In [1]:
import copy
import time 
import pickle
import numpy as np
import pandas as pd
import scipy as sc 
import scipy.io as scio
import scipy.sparse as sp
import matplotlib.pyplot as plt

from math import sqrt
from numpy import matmul
from MinTree import MinTree
from sklearn.utils.extmath import randomized_svd, squared_norm
from scipy.sparse import csc_matrix, coo_matrix, csr_matrix, lil_matrix

In [2]:
def norm(x):
    if sp.issparse(x):
        return sp.linalg.norm(x)
    return np.linalg.norm(x)


In [3]:
bio = scio.loadmat('BioSettings.mat')
dnew = bio['D_new'] # symmetric matrix that describe the dependency across the layers
do = bio['DO']  # the observed cross-layer dependencies
du = bio['DU']  # the complete cross-layer dependencies
g = bio['G']

# Compressed Sparse Column Matrix
g1 = lil_matrix(g[0][0][0][0][0])    # 6026 × 6026
g2 = lil_matrix(g[0][1][0][0][0])   # 25349 × 25349
g3 = lil_matrix(g[0][2][0][0][0])    # 4256 × 4256
c12 = du[0][0][0][0][0]  # 6026 × 25349
c13 = du[0][1][0][0][0]  # 6026 × 4256
c23 = du[0][2][0][0][0]  # 25349 × 4256
# 非1 entry转换为1
for a,b in zip(g1.nonzero()[0],g1.nonzero()[1]):
    g1[a,b] = 1
for a,b in zip(g2.nonzero()[0],g2.nonzero()[1]):
    g2[a,b] = 1
for a,b in zip(g3.nonzero()[0],g3.nonzero()[1]):
    g3[a,b] = 1
# Delete self-edges in g2

for idx in range(g2.shape[0]):
    g2[idx,idx] = 0
g1 = csc_matrix(g1)
g2 = csc_matrix(g2)
g3 = csc_matrix(g3)

In [9]:
g2

<25349x25349 sparse matrix of type '<class 'numpy.float64'>'
	with 306178 stored elements in Compressed Sparse Column format>

In [8]:
np.array(np.squeeze(c12.sum(axis=1).A))
np.array(np.squeeze(c12.sum(axis=0).A)).shape

(25349,)

In [11]:
AB = c12.copy()
B = g2.copy()
BC = c23.copy()

In [25]:
import easyFlowScope
import imp
imp.reload(easyFlowScope)
from easyFlowScope import FlowScope

In [26]:
fs = FlowScope([AB.tolil(),B.tolil(),BC.tolil()])

In [27]:
fs.run()

(6026, 25349) (25349, 25349) (25349, 4256)
6026 25349 25349
This is the cpu version of FlowScope
Start  greedy
s size:  56724
initial score of g:  3.680523235314858


KeyboardInterrupt: 

In [23]:
AB.T

<25349x6026 sparse matrix of type '<class 'numpy.float64'>'
	with 53735 stored elements in Compressed Sparse Row format>

In [20]:
deltas1 = np.array(np.squeeze(c12.sum(axis=1).A))
deltas2 = np.array(np.squeeze(c12.sum(axis=0).A)) + np.array(np.squeeze(c23.sum(axis=1).A))+np.array(np.squeeze(g2.sum(axis=0).A))
deltas3 = np.array(np.squeeze(c23.sum(axis=0).A))

In [19]:
np.array(np.squeeze(g2.sum(axis=0).A)).shape

(25349,)

In [22]:
g2

<25349x25349 sparse matrix of type '<class 'numpy.float64'>'
	with 306178 stored elements in Compressed Sparse Column format>

In [21]:
sum(deltas1) + sum(deltas2) + sum(deltas3)


236616.5